In [20]:
# First, use heuristics to get google perspectives that are not valid but are complete sentences.
import pandas as pd

perspective_db_path = "../data/database_output/re-step1/webapp_perspective.csv"

pdf = pd.read_csv(perspective_db_path).dropna()

# Only keep the google ones that (1) high agreement (2) Non-valid perspective
pdf = pdf[(pdf.pilot1_high_agreement == 1) & (pdf.pilot1_have_stance == 0) & (pdf.source == 'google')]
pdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4675 entries, 8137 to 20797
Data columns (total 7 columns):
id                       4675 non-null int64
source                   4675 non-null object
title                    4675 non-null object
pilot1_high_agreement    4675 non-null int64
similar_persps           4675 non-null object
more_than_two_tokens     4675 non-null int64
pilot1_have_stance       4675 non-null int64
dtypes: int64(4), object(3)
memory usage: 292.2+ KB


In [26]:
# Produce ids of the google perspectives that (1) high agreement (2) Non-valid perspective
from nltk import word_tokenize
import json

def complete_sentence_candidate(text, token_length=5):
    text = text.rstrip()
    toks = word_tokenize(text)
    legit = text[0].isupper() and toks[-1] == '.' and len(toks) > token_length
    return legit

pdf['legit_sentence'] = False
for idx, row in pdf.iterrows():
    if complete_sentence_candidate(row.title):
        pdf.at[idx, "legit_sentence"] = True

legit_sent_pdf = pdf[pdf.legit_sentence == True]
legit_sent_pdf.info()

index = legit_sent_pdf.id.unique().tolist()
indices_out_path = "../data/pilot17_making_the_dataset/indices/non_valid_high_agreement_google_persps.json"
with open(indices_out_path, 'w') as fout:
    json.dump(index, fout)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2427 entries, 8148 to 20792
Data columns (total 8 columns):
id                       2427 non-null int64
source                   2427 non-null object
title                    2427 non-null object
pilot1_high_agreement    2427 non-null int64
similar_persps           2427 non-null object
more_than_two_tokens     2427 non-null int64
pilot1_have_stance       2427 non-null int64
legit_sentence           2427 non-null bool
dtypes: bool(1), int64(4), object(3)
memory usage: 154.1+ KB


In [55]:
# 2. Produce ids of perspectives that (1) support or undermine (2) high agreement (aka > 0.5) (3) Original
# (4) have >= 15 other perspectives connected to the same claim 
perspective_db_path = "../data/database_output/re-step1/webapp_perspective.csv"
pdf = pd.read_csv(perspective_db_path).dropna()

# (1) support or undermine (2) high agreement (aka > 0.5) (3) Original
pdf = pdf[(pdf.pilot1_high_agreement == 1) & (pdf.pilot1_have_stance == 1) & (pdf.source != "google")]
pdf.info()

# Same criteria but in different table, for sanity check
re_step1_path = "../data/database_output/re-step1/webapp_restep1results.csv"
step1_df = pd.read_csv(re_step1_path)

step1_df = step1_df[(step1_df.p_i_3 > 0.5) & (step1_df.label_3.isin(["S", "U"])) & (step1_df.perspective_id < 8136)]
step1_df.info()

# Counts of how many perspectives each claim have
# For claims with > 10, get all perspectives ids
claim_persp_count_df = step1_df[['claim_id', 'perspective_id']].groupby(['claim_id']).count()
claim_persp_count_df = claim_persp_count_df[claim_persp_count_df.perspective_id > 10]
print(claim_persp_count_df)
# claim_ids = claim_persp_count_df.claim_id.unique()

# many_p_df = step1_df.loc[step1_df["claim_id"].isin(claim_ids)]
# many_p_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3985 entries, 0 to 8133
Data columns (total 7 columns):
id                       3985 non-null int64
source                   3985 non-null object
title                    3985 non-null object
pilot1_high_agreement    3985 non-null int64
similar_persps           3985 non-null object
more_than_two_tokens     3985 non-null int64
pilot1_have_stance       3985 non-null int64
dtypes: int64(4), object(3)
memory usage: 249.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3985 entries, 19372 to 23602
Data columns (total 12 columns):
id                        3985 non-null int64
claim_id                  3985 non-null int64
perspective_id            3985 non-null int64
vote_support              3985 non-null int64
vote_leaning_support      3985 non-null int64
vote_leaning_undermine    3985 non-null int64
vote_undermine            3985 non-null int64
vote_not_valid            3985 non-null int64
p_i_5                     3985 non-null floa

AttributeError: 'DataFrame' object has no attribute 'claim_id'